<a href="https://colab.research.google.com/github/AndresPautrat/Topicos_UPC/blob/main/ortools_SoftResctrictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

     |████████████████████████████████| 28.9MB 141kB/s 
     |████████████████████████████████| 1.3MB 45.1MB/s 
  Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0


In [ ]:
from ortools.sat.python import cp_model
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count
        solution_printer = VarArraySolutionPrinter([x,y,z])

In [ ]:
def generarSoluciones(valorSuma,minA,maxB,maxCosto):
  model = cp_model.CpModel()
  #variables
  a = model.NewIntVar(0,10,"a")
  b = model.NewIntVar(0,10,"b")
  #constraints
  ##hard : duras - estrictas
  model.Add(a + b == valorSuma)
  model.Add(b <= maxB)
  ##soft : suaves - flexibles
  #model.Add(a > minA)
  ##disjunctive relaxation
  ##=> (c == a > minA) 
  ##=> c' == [(c and z == 0) or (not c and z > 0)]
  ##=> c' == max(0,minA - a)
  z = model.NewIntVar(-10,10,"z")
  tmp = model.NewIntVar(-10,10,"z")
  model.Add(tmp == minA - a)
  model.AddMaxEquality(z,[0,tmp])
  model.Add(z <= maxCosto)
  #solver
  solver = cp_model.CpSolver()
  solution_printer = VarArraySolutionPrinter([a,b,z])
  solver.SearchForAllSolutions(model, solution_printer)
  print(solution_printer.solution_count(),"Solutions")

generarSoluciones(15,6,5,10)

a=10 b=5 z=0 
1 Solutions


In [ ]:

def generarSoluciones(valorSuma,minA,maxB,maxCosto):
  model = cp_model.CpModel()
  #variables
  a = model.NewIntVar(0,10,"a")
  b = model.NewIntVar(0,10,"b")
  #constraints
  ##hard : duras - estrictas
  model.Add(a + b == valorSuma)
  
  ##soft : suaves - flexibles
  #model.Add(a > minA)
  ##disjunctive relaxation
  ##=> (c == a > minA) 
  ##=> c' == [(c and z == 0) or (not c and z > 0)]
  ##=> c' == max(0,minA - a)
  z = [model.NewIntVar(-10,10,"z"+str(i)) for i in range(2)]

  tmp1 = model.NewIntVar(-10,10,"t1")
  model.Add(tmp1 == minA - a)
  model.AddMaxEquality(z[0],[0,tmp1])
  #model.Add(z <= maxCosto)

  #model.Add(b <= maxB)
  ##soft => (c == b <= maxB)   
  ##=> c' == [(c and z == 0) or (not c and z > 0)]
  ##=> c' == max(0,b - maxB)
  tmp2 = model.NewIntVar(-10,10,"t2")
  model.Add(tmp2 == b - maxB)
  model.AddMaxEquality(z[1],[0,tmp2])
  #model.Add(z <= maxCosto)
  
  model.Add(sum(z) <= maxCosto)

  #solver
  solver = cp_model.CpSolver()
  solution_printer = VarArraySolutionPrinter([a,b] + z)
  solver.SearchForAllSolutions(model, solution_printer)
  print(solution_printer.solution_count(),"Solutions")

##a in [0..10]
##b in [0..10]
##a + b == 15
##a >= 6
##b <= 4
generarSoluciones(15,6,4,10)

a=5 b=10 z0=1 z1=6 
a=10 b=5 z0=0 z1=1 
a=8 b=7 z0=0 z1=3 
a=7 b=8 z0=0 z1=4 
a=6 b=9 z0=0 z1=5 
a=9 b=6 z0=0 z1=2 
6 Solutions
